In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
from matplotlib import pyplot as plt

In [5]:
def readBadgesData(path):
    data = {
        'plus': [],
        'firstname': [],
        'lastname': []
    }
    
    with open(path, 'r') as f:
        line = f.readline()
        while line:
            entry = line.split()
            if len(entry) == 3 and len(entry[1]) >= 3 and len(entry[-1]) >= 3:
                data['plus'].append(True if entry[0] == '+' else False)
                data['firstname'].append(entry[1].lower())
                data['lastname'].append(entry[-1].lower())
            
            line = f.readline()
    
    return pd.DataFrame(data)

df = readBadgesData('./badges.txt')

In [6]:
df.describe()

,plus,firstname,lastname
count,219,219,219
unique,2,182,215
top,True,david,gordon
freq,160,9,2


In [7]:
df.sample(10)

,plus,firstname,lastname
180,True,mark,schwabacher
93,True,wolfgang,janko
21,True,justin,boyan
74,True,marko,grobelnik
35,True,jeffery,clouse
201,True,tatsuo,unemi
137,False,steven,minton
14,True,george,berg
26,True,bill,byrne
113,True,martin,kummer


In [8]:
from string import ascii_lowercase

def getLetterCounts(name):
    letters_list = list(0 for _ in range(len(ascii_lowercase)))
    for letter in name:
        if letter in ascii_lowercase:
            letters_list[ord(letter) - ord('a')] += 1
    
    return letters_list

getLetterCounts('lol') 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [24]:
from sklearn.tree import DecisionTreeClassifier

firstnameLetterCounts = np.array(list(df['firstname'].apply(getLetterCounts)))
lastnameLetterCounts = np.array(list(df['lastname'].apply(getLetterCounts)))
firstnameLen = df['firstname'].apply(len).values.reshape(-1, 1)
lastnameLen = df['lastname'].apply(len).values.reshape(-1, 1)

train_data = np.c_[firstnameLetterCounts, lastnameLetterCounts, firstnameLen, lastnameLen]
feature_names = (['%s in firstname' % l for l in ascii_lowercase] 
                 + ['%s in lastname' % l for l in ascii_lowercase]
                 + ['firstname length', 'lastname length'])
        
train_labels = df['plus'].values

# clf_tree = DecisionTreeClassifier(criterion='entropy', max_depth=3)
clf_tree = DecisionTreeClassifier(max_depth=5)
clf_tree.fit(train_data, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [25]:
# from https://github.com/Yorko/mlcourse.ai/blob/master/jupyter_english/topic03_decision_trees_kNN/topic3_decision_trees_kNN.ipynb
import pydotplus #pip install pydotplus
from sklearn.tree import export_graphviz

def tree_graph_to_png(tree, feature_names, png_file_to_save):
    tree_str = export_graphviz(tree, feature_names=feature_names, 
                                     filled=True, out_file=None)
    graph = pydotplus.graph_from_dot_data(tree_str)  
    graph.write_png(png_file_to_save)

In [26]:
tree_graph_to_png(tree=clf_tree, feature_names=feature_names, 
                  png_file_to_save='./tree1.png')

![](./tree1.png)